# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы . Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
# импортируем необходимые для исследования библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from tqdm import tqdm

# открываем и читаем файл
df = pd.read_csv('/datasets/users_behavior.csv')
display(df)
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


##### Вывод

Данные о абонентах состоят из 5 колонок:
   - сalls — количество звонков,
   - minutes — суммарная длительность звонков в минутах,
   - messages — количество sms-сообщений,
   - mb_used — израсходованный интернет-трафик в Мб,
   - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
   
Количество абонентов 3214. Предобработка данных не требуется.
   

In [2]:
# проверяем мультиколлинеарность модели
df.corr(method='spearman')

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.978684,0.155032,0.253886,0.160549
minutes,0.978684,1.000000,0.153784,0.248818,0.159991
messages,0.155032,0.153784,1.000000,0.141999,0.106537
mb_used,0.253886,0.248818,0.141999,1.000000,0.154989
is_ultra,0.160549,0.159991,0.106537,0.154989,1.000000


## Разбейте данные на выборки

In [3]:
# извлекаем признаки
features = df.drop(['is_ultra'], axis = 1)
# извлекаем целевой признак
target = df['is_ultra']

Разделим данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Валидационная выборка нужна для проверки наших моделей во время их точной настройки, тестовая — в качестве окончательного датасета для сравнения моделей и выбора лучшей.

In [4]:
# выделяем 60% данных для обучающей выборки, остальные данные для тестовой и валидационной.
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, test_size = 0.4, 
                                                                            random_state = 12345)
# разделяем тестовую и валидационную выборку часть на равные части 
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, test_size = 0.5, 
                                                                            random_state = 12345)


In [5]:
# размер обучающей выборки
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


In [6]:
# размер валидационной выборки
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


In [7]:
# размер тестовой выборки
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


##### Вывод

Разбили данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Валидационная выборка нужна для проверки наших моделей во время их точной настройки, тестовая — в качестве окончательного датасета для сравнения моделей и выбора лучшей.

## Исследуйте модели

#### DecisionTreeRegressor

In [8]:
# вводим необходимые переменные 
best_model_dtc = None
best_result_dtc = 0
best_depth_dtc = 0

for depth in tqdm(range (1, 11)):
    # инициализируеем модель
    model_dtc = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    # обучаем модель на обучающей выборке
    model_dtc.fit(features_train, target_train)
    # находим предсказания на валидационной выборке
    predictions_valid_dtc = model_dtc.predict(features_valid)
    # находим долю правильных ответов
    result_dtc = accuracy_score(target_valid, predictions_valid_dtc)
    # находим лучшую долю правильных ответов взависимости от максимальной "глубины дерева"
    if result_dtc > best_result_dtc:
        best_model_dtc = model_dtc
        best_result_dtc = result_dtc
        best_depth_dtc = depth
print('Accuracy наилучшей модели на валидационной выборке = ',best_result_dtc, 'при "глубине дерева"', best_depth_dtc,'.')        
    


100%|██████████| 10/10 [00:00<00:00, 86.13it/s]

Accuracy наилучшей модели на валидационной выборке =  0.7853810264385692 при "глубине дерева" 3 .


#### RandomForestClassifier

In [9]:
# при количестве деревьев от 1 до 10, с глубиной от 1 до 10
# вводим необходимые переменные 
best_model_rfc = None
best_result_rfc = 0
best_depth_rfc = 0
best_est_rfc = 0

for est in  tqdm(range (1, 11)):
    for depth in range(1, 11):
        # инициализируем модель
        model_rfc = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        # обучаем модель на обучающей выборке
        model_rfc.fit(features_train, target_train)
        # находим предсказания на валидационной выборке
        predictions_valid_rfc = model_rfc.predict(features_valid)
        # находим долю правильных ответов
        result_rfc = accuracy_score(target_valid, predictions_valid_rfc)
        # находим лучшую долю правильных ответов взависимости от количества деревьев и максимальной "глубины дерева"
        if result_rfc > best_result_rfc:
            best_model_rfc = model_rfc
            best_result_rfc = result_rfc
            best_depth_rfc = depth
            best_est_rfc = est
            
print('Accuracy наилучшей модели на валидационной выборке =',best_result_dtc, ', количество деревьев :',best_est_rfc, ', при "глубине дерева":', best_depth_dtc,'.')        
    


100%|██████████| 10/10 [00:04<00:00,  2.19it/s]

Accuracy наилучшей модели на валидационной выборке = 0.7853810264385692 , количество деревьев : 8 , при "глубине дерева": 3 .


In [10]:
# при количестве деревьев от 1 до 30, с глубиной от 1 до 10
# вводим необходимые переменные 
best_model_rfc = None
best_result_rfc = 0
best_depth_rfc = 0
best_est_rfc = 0

for est in  tqdm(range (1, 31)):
    for depth in range(1, 11):
        # инициализируем модель
        model_rfc = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        # обучаем модель на обучающей выборке
        model_rfc.fit(features_train, target_train)
        # находим предсказания на валидационной выборке
        predictions_valid_rfc = model_rfc.predict(features_valid)
        # находим долю правильных ответов
        result_rfc = accuracy_score(target_valid, predictions_valid_rfc)
        # находим лучшую долю правильных ответов взависимости от количества деревьев и максимальной "глубины дерева"
        if result_rfc > best_result_rfc:
            best_model_rfc = model_rfc
            best_result_rfc = result_rfc
            best_depth_rfc = depth
            best_est_rfc = est
            
print('Accuracy наилучшей модели на валидационной выборке =',best_result_dtc, ', количество деревьев :',best_est_rfc, ', при "глубине дерева":', best_depth_dtc,'.')        
    


100%|██████████| 30/30 [00:23<00:00,  1.28it/s]

Accuracy наилучшей модели на валидационной выборке = 0.7853810264385692 , количество деревьев : 26 , при "глубине дерева": 3 .


In [11]:
# при количестве деревьев от 1 до 10, с глубиной от 1 до 30
# вводим необходимые переменные 
best_model_rfc = None
best_result_rfc = 0
best_depth_rfc = 0
best_est_rfc = 0

for est in  tqdm(range (1, 11)):
    for depth in range(1, 31):
        # инициализируем модель
        model_rfc = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        # обучаем модель на обучающей выборке
        model_rfc.fit(features_train, target_train)
        # находим предсказания на валидационной выборке
        predictions_valid_rfc = model_rfc.predict(features_valid)
        # находим долю правильных ответов
        result_rfc = accuracy_score(target_valid, predictions_valid_rfc)
        # находим лучшую долю правильных ответов взависимости от количества деревьев и максимальной "глубины дерева"
        if result_rfc > best_result_rfc:
            best_model_rfc = model_rfc
            best_result_rfc = result_rfc
            best_depth_rfc = depth
            best_est_rfc = est
            
print('Accuracy наилучшей модели на валидационной выборке =',best_result_dtc, ', количество деревьев :',best_est_rfc, ', при "глубине дерева":', best_depth_dtc,'.')        
    


100%|██████████| 10/10 [00:06<00:00,  1.64it/s]

Accuracy наилучшей модели на валидационной выборке = 0.7853810264385692 , количество деревьев : 8 , при "глубине дерева": 3 .


#### Logistic Regression

In [12]:
# инициализируем модель
model_lr = LogisticRegression(random_state=12345, solver='lbfgs')
# обучаем модель на обучающей выборке
model_lr.fit(features_train, target_train)
# находим предсказания на валидационной выборке
predictions_valid_lr = model_lr.predict(features_valid)
# находим долю правильных ответов
accuracy_lr = accuracy_score(predictions_valid_lr, target_valid)

print('Accuracy наилучшей модели на валидационной выборке =', accuracy_lr,'.')        
       

Accuracy наилучшей модели на валидационной выборке = 0.7107309486780715 .


##### Вывод

Наилучший и равный результат Accuracy получили модели решающего дерева и случайного леса = 0.7853810264385692. По производительности модель решающего дерева быстрее, чем случайного леса  173.77it/s против 6.03it/s. Лучшая модель случайного леса с количеством деревьев 8 и глубиной деревьев 3. Результат логической регрессии немногим хуже 0.7107309486780715.

## Проверьте модель на тестовой выборке

Модель решающего дерева более эффективна, чем модель случайного леса 173.77it/s против 6.03it/s, поэтому проверим её на тестовой выборке.

In [13]:
# инициализируем модель
model_dtc = DecisionTreeClassifier(random_state = 12345, max_depth = 3)
# обучаем модель на обучающей выборке
model_dtc.fit(features_train, target_train)
# находим предсказания на тестовой выборке
predictions_test_dtc = model_dtc.predict(features_test)
# находим долю правильных ответов на тестовой выборке
result_test_dtc = accuracy_score(target_test, predictions_test_dtc)

print('Accuracy наилучшей модели на тестовой выборке = ',result_test_dtc,'.')  

Accuracy наилучшей модели на тестовой выборке =  0.7791601866251944 .


##### Вывод

На тестовой выборке accuracy показала результат 0.7791601866251944, что удовлетворяет поставленной задаче, что нужно довести долю правильных ответов по крайней мере до 0.75.

## (бонус) Проверьте модели на адекватность

Чтобы оценить адекватность модели будем использовать DummyClassifier 

In [14]:
# инициализируем модель с "most_frequent",  предсказывающая наиболее частую метку в обучающем наборе
model_ad = DummyClassifier(strategy='most_frequent', random_state=12345)
# обучаем модель на обучающей выборке
model_ad = model_ad.fit(features_train, target_train)
# находим долю правильных ответов валидационной выборке
result_ad = model_ad.score(features_valid, target_valid)

print('Accuracy работоспособности модели на валидационной выборке = ', result_ad)

Accuracy работоспособности модели на валидационной выборке =  0.7060653188180405


##### Вывод

С данным Accuracy можем считать модель работоспособной (адекватной).

## Общий вывод

- Данные о абонентах состоят из 5 колонок:
  - сalls — количество звонков,
  - minutes — суммарная длительность звонков в минутах,
  - messages — количество sms-сообщений,
  - mb_used — израсходованный интернет-трафик в Мб,
  - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
  
- Количество исследуемых абонентов 3214. 
- Данные были разбиты на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

- Наилучший и равный результат Accuracy получили модели решающего дерева и случайного леса = 0.7853810264385692. По производительности модель решающего дерева быстрее, чем случайного леса 173.77it/s против 6.03it/s. Лучшая модель случайного леса при подборе гиперпараметров с количеством деревьев 8 и глубиной деревьев 3. Результат логической регрессии немногим хуже 0.7107309486780715.

- На тестовой выборке accuracy показала результат 0.7791601866251944, что удовлетворяет поставленной задаче, что нужно довести долю правильных ответов по крайней мере до 0.75.

- При проверке модели на адекватность Accuracy = 0.7060653188180405, что позволяет считать модель работоспособной (адекватной).